Policy Responses to COVID-19 Dataset
- Authors of the dataset provided an excel containing all the dataset and brief summary of the fields. In total, 40K data is presented
- Sources of each document the authors linked is also present in the dataset



---



Approach:
Only keep relevant column in the dataset (Index, Excerpt, Date, Bias Type, Source, Normative Framing)

  Column Details:
  - ID is 0-N , where N is size of the dataset
  - Excerpt of policy will be based on "Notes" column
  - Bias Type is a new field
  - Source is a field where url of the actual policy is taken
  - Normative framing (Implicit or explicit) is a new field


---



1. Remove fields where the field of "Notes" are not acoording to criteria:
- Notes field are empty
- Exends from previous policy
- Has 9 words or less (length of words help with determining bias)

2. Add ID Column

3. Rename 'effective_date' as Date




Filtering the fields result with a total of 7,737 Data

In [34]:
# Import necessary library
import pandas as pd
import requests


In [36]:

# Load data
df = pd.read_excel('/content/sprc19_v6.xlsx')

# Drop rows with missing 'notes'
df = df.dropna(subset=['notes'])

# Define exclusion patterns
patterns = [
    r'extend(ed)? from',
    r'\b(extension of|extend|extends)\b',
    r'\bmodifies\b.*\border\b',
    r'\b(extending|extedns)\b'
]

# Apply filters to remove matching rows in 'notes'
for pattern in patterns:
    df = df[~df['notes'].str.contains(pattern, case=False, na=False, regex=True)]

# Remove rows where 'notes' has 9 or fewer words
df = df[df['notes'].str.split().str.len() > 9]

# Drop duplicate 'notes', keeping first occurrence
df = df.drop_duplicates(subset='notes')


# Rename 'effective_date' to 'Date' Column
df = df.rename(columns={'effective_date': 'Date'})


# Rename 'notes' column as 'excerpt'
df = df.rename(columns={'notes': 'Excerpt'})

# Insert ID Column
df.insert(0, 'ID', range(1, len(df) + 1))


# Save cleaned data
df.to_excel('/content/sprc19_v6.xlsx', index=False)


<ipython-input-36-2969092ce0ad>:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['notes'].str.contains(pattern, case=False, na=False, regex=True)]


4. Find source url for each policy by using API Calls

In [37]:


# Load Excel file
df = pd.read_excel('/content/sprc19_v6.xlsx')

# setup filename
filename_col = 'sourcefile'

# Get file list from Dataverse API
api_url = "https://dataverse.harvard.edu/api/datasets/:persistentId/versions/:latest/files?persistentId=doi:10.7910/DVN/GJAUGE"
files = requests.get(api_url).json()['data']

# Build mapping from filename to fileId
file_map = {f['label']: f['dataFile']['id'] for f in files}

# Fill column
def make_url(row):
    filename = row[filename_col]
    fileid = file_map.get(filename)
    if fileid:
        return f"https://dataverse.harvard.edu/file.xhtml?fileId={fileid}"
    return ""

df['Source'] = df.apply(make_url, axis=1)
df.to_excel('/content/sprc19_v6.xlsx', index=False)


3. Only Keep Index, Excerpt, Date Column. Then add new column:
Bias Type, Source, Normative Framing

In [38]:


# Load Excel file
df = pd.read_excel('/content/sprc19_v6.xlsx')

# Keep only ID, excerpt, and date columns
df = df[['ID', 'Excerpt', 'Date', 'Source']]

# Add new empty columns
df['Bias_type'] = ''
df['Normative_framing'] = ''

# Reorder columns
df = df[['ID', 'Excerpt', 'Date', 'Bias_type', 'Normative_framing', 'Source']]

# Save cleaned DataFrame
df.to_excel('/content/sprc19_v6_formatted.xlsx', index=False)

<ipython-input-38-03ec6258c696>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bias_type'] = ''
<ipython-input-38-03ec6258c696>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Normative_framing'] = ''
